# Deutsche Bahn data scraping

## Basics

In [2]:
import requests
import pandas as pd
import csv
import pickle
import numpy as np

In [3]:
url_parking = 'https://apis.deutschebahn.com/db-api-marketplace/apis/parking-information/db-bahnpark/v2/'
url_ris_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/'
url_rw_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/api.railway-stations.org/photoStationById/'
url_facility_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/fasta/v2/stations/'
url_parking_facilities = 'https://apis.deutschebahn.com/db-api-marketplace/apis/parking-information/db-bahnpark/v2/parking-facilities'

client_id = '22a834e6e490f430dc9250bc13e7fba0'
client_secret = '0f294b9c1f1c72f834dbc43cdf33e42d'
limit = 500 # this is the default

In [4]:
headers = {
    'DB-Client-Id': client_id,
    'DB-Api-Key': client_secret,
}

In [5]:
# response = requests.get(url_parking + 'parking-facilities', headers=headers)
# print(response.json())

## Stations

This will return all available stations [Bahnhöfe].

See [here](https://developers.deutschebahn.com/db-api-marketplace/apis/product/ris-stations/api/ris-stations#/RISStations_1134/operation/%2Fstations/get) for documentation.

In [5]:
offset = 0
total = 9999
jsons = []

while (offset < total):
    response = requests.get(url_ris_stations + f'stations?offset={offset}&limit={limit}', headers=headers)
    json = response.json()
    offset += 500
    total = json['total']
    jsons.append(json)

KeyError: 'total'

In [7]:
jsons[0]

{'offset': 0,
 'limit': 500,
 'total': 5690,
 'stations': [{'stationID': '1',
   'names': {'DE': {'name': 'Aachen Hbf'}},
   'metropolis': {},
   'address': {'street': 'Bahnhofstr.',
    'houseNumber': '2a',
    'postalCode': '52064',
    'city': 'Aachen',
    'state': 'Nordrhein-Westfalen',
    'country': 'DE'},
   'stationCategory': 'CATEGORY_2',
   'availableTransports': [],
   'availableLocalServices': [],
   'owner': {'name': 'DB S&S',
    'organisationalUnit': {'id': 4,
     'name': 'RB West',
     'nameShort': 'RB West'}},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin',
   'position': {'longitude': 6.091499, 'latitude': 50.7678},
   'validFrom': '2019-01-01T00:00:00Z'},
  {'stationID': '1000',
   'names': {'DE': {'name': 'Burkhardswalde-Maxen'}},
   'metropolis': {},
   'address': {'street': 'Gesundbrunnen',
    'houseNumber': '60c',
    'postalCode': '01809',
    'city': 'Müglitztal-Burkhardswalde',
    'state': 'Sachsen',
    'country': 'DE'},
   'stationCategory': 'CA

In [8]:
stations_list = []

for json in jsons:
    for entry in json['stations']:
        stations_list.append(
            {
                "id": entry['stationID'],
                "name": entry['names'].get('DE', {}).get('name', ''),
                "metropolis": entry.get('metropolis', ''),
                "street": entry['address'].get('street', ''),
                "houseNumber": entry['address'].get('houseNumber', ''),
                "postalCode": entry['address'].get('postalCode', ''),
                "city": entry['address'].get('city', ''),
                "state": entry['address'].get('state', ''),
                "country": entry['address'].get('country', ''),
                "stationCategory": entry.get('stationCategory', ''),
                "owner": entry['owner'].get('name', ''),
                "organisationalUnit": entry['owner'].get('organisationalUnit', {}).get('name', ''),
                "countryCode": entry['countryCode'],
                "latitude": entry.get('position', {}).get('latitude', ''),
                "longitude": entry.get('position', {}).get('longitude', ''),
                "timeZone": entry['timeZone']
            }
        )


In [37]:
len(stations_list)
df_stations = pd.DataFrame(stations_list)
df_stations = df_stations.replace('', np.nan)

In [36]:
df_stations.isna().sum()

id                      0
name                    0
metropolis              0
street                  8
houseNumber           893
postalCode              7
city                    4
state                   0
country                 0
stationCategory        12
owner                   0
organisationalUnit      0
countryCode             0
latitude              282
longitude             282
timeZone                0
dtype: int64

In [38]:
output = open('stations.pkl', 'wb')
pickle.dump(df_stations, output)
output.close()

In [32]:
csv_file_path = 'stations.csv'

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=stations_list[0].keys())
    
    # Write header
    writer.writeheader()
    
    # Write data rows
    writer.writerows(stations_list)

## Stop-places

In [8]:
pkl_file = open('stations.pkl', 'rb')
stations_list = pickle.load(pkl_file)
pkl_file.close()

In [7]:
stations_list

,id,name,metropolis,street,houseNumber,postalCode,city,state,country,stationCategory,owner,organisationalUnit,countryCode,latitude,longitude,timeZone
0,1,Aachen Hbf,{},Bahnhofstr.,2a,52064,Aachen,Nordrhein-Westfalen,DE,CATEGORY_2,DB S&S,RB West,DE,50.767800,6.091499,Europe/Berlin
1,1000,Burkhardswalde-Maxen,{},Gesundbrunnen,60c,01809,Müglitztal-Burkhardswalde,Sachsen,DE,CATEGORY_7,DB S&S,RB Südost,DE,50.925146,13.838369,Europe/Berlin
2,1001,Burkhardtsdorf,{},Bahnhofstraße,NaN,09235,Burkhardtsdorf,Sachsen,DE,CATEGORY_6,DB Regio-Netze,Erzgebirgsbahn (EGB),DE,NaN,NaN,Europe/Berlin
3,1002,Bürstadt,{},Bahnhofsallee,17,68642,Bürstadt,Hessen,DE,CATEGORY_6,DB S&S,RB Mitte,DE,49.645769,8.458188,Europe/Berlin
4,1005,Buschow,{},Bahnhofstr.,28,14715,Märkisch Luch OT Buschow,Brandenburg,DE,CATEGORY_6,DB S&S,RB Ost,DE,52.592203,12.628996,Europe/Berlin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5685,995,Burgstädt,{},Bahnhofstr.,1,09217,Burgstädt,Sachsen,DE,CATEGORY_6,DB S&S,RB Südost,DE,50.915817,12.812707,Europe/Berlin
5686,996,Burgstall (Murr),{},Bahnhofstr.,1,71576,Burgstetten,Baden-Württemberg,DE,CATEGORY_6,DB S&S,RB Südwest,DE,48.928647,9.369932,Europe/Berlin
5687,997,Steinfurt-Burgsteinfurt,{},Bahnhofsplatz,6,48565,Steinfurt-Burgsteinfurt,Nordrhein-Westfalen,DE,CATEGORY_6,DB S&S,RB West,DE,52.147384,7.329340,Europe/Berlin
5688,998,Burgthann,{},Bahnhofstr.,40,90559,Burgthann,Bayern,DE,CATEGORY_5,DB S&S,RB Süd,DE,49.342474,11.309307,Europe/Berlin


In [8]:
stations_list['id'][0]

'1'

## Local Services

In [30]:
local_services_jsons = []
keyType = 'STATION_ID' #  (Stationsdatenbank number)

for index in stations_list.index:
    response = requests.get(url_ris_stations + f'local-services/by-key?keyType={keyType}&key={stations_list["id"][index]}', headers=headers)
    json = response.json()
    local_services_jsons.append(json)

In [22]:
local_services_jsons[:1]

[{'localServices': [{'localServiceID': 'MOBILE_TRAVEL_SERVICE:1',
    'stationID': '1',
    'type': 'MOBILE_TRAVEL_SERVICE',
    'openingHours': 'Mo-Su 06:15-22:30;PH 06:15-22:30'},
   {'localServiceID': 'TRIPLE_S_CENTER:1',
    'stationID': '1',
    'externalID': '15',
    'type': 'TRIPLE_S_CENTER',
    'name': 'Duisburg Hbf',
    'contact': {'phoneNumbers': [{'type': 'BUSINESS',
       'number': '0203/30171055'}]},
    'openingHours': 'Mo-Su 00:00-24:00;PH 00:00-24:00'},
   {'localServiceID': 'RAILWAY_MISSION:1',
    'stationID': '1',
    'type': 'RAILWAY_MISSION'},
   {'localServiceID': 'HANDICAPPED_TRAVELLER_SERVICE:1',
    'stationID': '1',
    'type': 'HANDICAPPED_TRAVELLER_SERVICE',
    'description': 'Ja, um Voranmeldung unter 030 65 21 28 88 (Ortstarif) wird gebeten'},
   {'localServiceID': 'LOCKER:1', 'stationID': '1', 'type': 'LOCKER'},
   {'localServiceID': 'WIFI:1', 'stationID': '1', 'type': 'WIFI'},
   {'localServiceID': 'CAR_PARKING:1',
    'stationID': '1',
    'type': 

In [31]:
localservices_list = []

for json in local_services_jsons:
    for entry in json.get('localServices', {}):
        localservices_list.append(
            {
                "id": entry['stationID'],
                "name": entry.get('name'),
                "description": entry.get('description'),
                "openingHours": entry.get('openingHours'),
                "latitude": entry.get('position', {}).get('latitude', ''),
                "longitude": entry.get('position', {}).get('longitude', ''),
                "type": entry.get('type')            
            }
        )

In [32]:
df_localservices = pd.DataFrame(localservices_list)
df_localservices

,id,name,description,openingHours,latitude,longitude,type
0,1,None,None,Mo-Su 06:15-22:30;PH 06:15-22:30,,,MOBILE_TRAVEL_SERVICE
1,1,Duisburg Hbf,None,Mo-Su 00:00-24:00;PH 00:00-24:00,,,TRIPLE_S_CENTER
2,1,None,None,None,,,RAILWAY_MISSION
3,1,None,"Ja, um Voranmeldung unter 030 65 21 28 88 (Ort...",None,,,HANDICAPPED_TRAVELLER_SERVICE
4,1,None,None,None,,,LOCKER
...,...,...,...,...,...,...,...
14140,6110,None,None,None,,,BICYCLE_PARKING
14141,6112,None,None,None,,,CAR_PARKING
14142,6112,Saarbrücken Hbf,None,Mo-Su 00:00-24:00;PH 00:00-24:00,,,TRIPLE_S_CENTER
14143,6113,Saarbrücken Hbf,None,Mo-Su 00:00-24:00;PH 00:00-24:00,,,TRIPLE_S_CENTER


In [33]:
output = open('localservices.pkl', 'wb')
pickle.dump(df_localservices, output)
output.close()

## Stop Places

In [14]:
stopplaces_jsons = []
keyType = 'STADA' #  (Stationsdatenbank number)

for index in stations_list.index:
    response = requests.get(url_ris_stations + f'stop-places/by-key?keyType={keyType}&key={stations_list["id"][index]}', headers=headers)
    json = response.json()
    stopplaces_jsons.append(json)

In [16]:
stopplaces_jsons

[{'stopPlaces': [{'evaNumber': '8000001',
    'stationID': '1',
    'names': {'DE': {'nameLong': 'Aachen Hbf',
      'speechLong': 'Aachen Hbf',
      'speechShort': 'Aachen Hbf'}},
    'availableTransports': ['INTERCITY_TRAIN',
     'CITY_TRAIN',
     'BUS',
     'REGIONAL_TRAIN',
     'INTER_REGIONAL_TRAIN',
     'HIGH_SPEED_TRAIN'],
    'transportAssociations': ['AAV', 'VRS'],
    'countryCode': 'DE',
    'state': 'NW',
    'municipalityKey': '05334002',
    'timeZone': 'Europe/Berlin',
    'position': {'longitude': 6.091499, 'latitude': 50.7678}}]},
 {'stopPlaces': [{'evaNumber': '8011298',
    'stationID': '1000',
    'names': {'DE': {'nameLong': 'Burkhardswalde-Maxen',
      'speechLong': 'Burkhardswalde-Maxen',
      'speechShort': 'Burkhardswalde-Maxen'}},
    'metropolis': {'DE': 'Müglitztal-Burkhardswa'},
    'availableTransports': ['REGIONAL_TRAIN'],
    'transportAssociations': ['VVO'],
    'countryCode': 'DE',
    'state': 'SN',
    'municipalityKey': '14628250',
    'time

In [15]:
stopplaces_list = []

for json in stopplaces_jsons:
    for entry in json.get('stopPlaces', {}):
        stopplaces_list.append(
            {
                "id": entry['stationID'],
                "name": entry['names'].get('DE', {}).get('nameLong', ''),
                "availableTransports": entry.get('availableTransports', []),
                "transportAssociations": entry.get('transportAssociations', []),
                "countryCode": entry['countryCode'],
                "state": entry.get('state', ''),
                "timeZone": entry['timeZone'],
                "latitude": entry.get('position', {}).get('latitude', ''),
                "longitude": entry.get('position', {}).get('longitude', '')                
            }
        )

In [16]:
df_stopplaces = pd.DataFrame(stopplaces_list)
df_stopplaces = df_stopplaces.replace('', np.nan)

In [17]:
output = open('stopplaces_new.pkl', 'wb')
pickle.dump(df_stopplaces, output)
output.close()

In [42]:
csv_file_path = 'stopplaces.csv'

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=stopplaces_list[0].keys())
    
    # Write header
    writer.writeheader()
    
    # Write data rows
    writer.writerows(stopplaces_list)

## Pictures of stations

In [6]:
pkl_file = open('stations.pkl', 'rb')
df_stations = pickle.load(pkl_file)
pkl_file.close()

In [12]:
rw_pictures = {}
base_url = 'https://api.railway-stations.org/photos'

for entry in df_stations['id']:
    try:
        response = requests.get(url_rw_stations + f'DE/{entry}', headers=headers)
        json = response.json()
        for station in json['stations']:
            images = []
            for photo in station['photos'][:1]:
                images.append(base_url + photo['path'])
            rw_pictures.setdefault(entry, images)
    except:
        pass

rw_pictures

{'1': ['https://api.railway-stations.org/photos/de/1_1.jpg'],
 '1000': ['https://api.railway-stations.org/photos/de/1000.jpg'],
 '1001': ['https://api.railway-stations.org/photos/de/1001_0.jpg'],
 '1002': ['https://api.railway-stations.org/photos/de/1002_1.jpg'],
 '1005': ['https://api.railway-stations.org/photos/de/1005_1.jpg'],
 '1006': ['https://api.railway-stations.org/photos/de/1006_0.jpg'],
 '1008': ['https://api.railway-stations.org/photos/de/1008.jpg'],
 '1009': ['https://api.railway-stations.org/photos/de/1009.jpg'],
 '1011': ['https://api.railway-stations.org/photos/de/1011.jpg'],
 '1012': ['https://api.railway-stations.org/photos/de/1012.jpg'],
 '1013': [],
 '1014': ['https://api.railway-stations.org/photos/de/1014_1.jpg'],
 '1015': ['https://api.railway-stations.org/photos/de/1015.jpg'],
 '1016': ['https://api.railway-stations.org/photos/de/1016.jpg'],
 '1017': ['https://api.railway-stations.org/photos/de/1017.jpg'],
 '1018': ['https://api.railway-stations.org/photos/de/101

In [59]:
rw_pictures

{'1': ['https://api.railway-stations.org/photos/de/1_1.jpg'],
 '1000': ['https://api.railway-stations.org/photos/de/1000.jpg'],
 '1001': ['https://api.railway-stations.org/photos/de/1001_0.jpg'],
 '1002': ['https://api.railway-stations.org/photos/de/1002_1.jpg'],
 '1005': ['https://api.railway-stations.org/photos/de/1005_1.jpg'],
 '1006': ['https://api.railway-stations.org/photos/de/1006_0.jpg'],
 '1008': ['https://api.railway-stations.org/photos/de/1008.jpg'],
 '1009': ['https://api.railway-stations.org/photos/de/1009.jpg'],
 '1011': ['https://api.railway-stations.org/photos/de/1011.jpg'],
 '1012': ['https://api.railway-stations.org/photos/de/1012.jpg'],
 '1013': [],
 '1014': ['https://api.railway-stations.org/photos/de/1014_1.jpg'],
 '1015': ['https://api.railway-stations.org/photos/de/1015.jpg'],
 '1016': ['https://api.railway-stations.org/photos/de/1016.jpg'],
 '1017': ['https://api.railway-stations.org/photos/de/1017.jpg'],
 '1018': ['https://api.railway-stations.org/photos/de/101

In [13]:
output = open('station_images.pkl', 'wb')
pickle.dump(rw_pictures, output)
output.close()

## Facility status

In [4]:
pkl_file = open('stations.pkl', 'rb')
df_stations = pickle.load(pkl_file)
pkl_file.close()

In [19]:
facility_json = []

for entry in df_stations['id']:
    try:
        response = requests.get(url_facility_stations + f'/{entry}', headers=headers)
        json = response.json()
        facility_json.append(json)
    except:
        pass

facility_json

[{'facilities': [{'description': 'zu Gleis 1',
    'equipmentnumber': 10318681,
    'geocoordX': 6.0911305,
    'geocoordY': 50.7680045,
    'operatorname': 'DB Station&Service',
    'state': 'ACTIVE',
    'stateExplanation': 'available',
    'stationnumber': 1,
    'type': 'ELEVATOR'},
   {'description': 'zu Gleis 2/3',
    'equipmentnumber': 10355943,
    'geocoordX': 6.0912217,
    'geocoordY': 50.767839,
    'operatorname': 'DB Station&Service',
    'state': 'ACTIVE',
    'stateExplanation': 'available',
    'stationnumber': 1,
    'type': 'ELEVATOR'},
   {'description': 'zu Gleis 6/7',
    'equipmentnumber': 10355942,
    'geocoordX': 6.09134,
    'geocoordY': 50.767629,
    'operatorname': 'DB Station&Service',
    'state': 'ACTIVE',
    'stateExplanation': 'available',
    'stationnumber': 1,
    'type': 'ELEVATOR'},
   {'description': 'zu Gleis 8/9',
    'equipmentnumber': 10355941,
    'geocoordX': 6.0914006,
    'geocoordY': 50.7675157,
    'operatorname': 'DB Station&Service

In [20]:
facilities_list = []

for json in facility_json:
    for entry in json.get('facilities', {}):
        facilities_list.append(
            {
                "id": entry['stationnumber'],
                "description": entry.get('description'),
                "operatorname": entry.get('operatorname'),
                "state": entry.get('state'),
                "stateExplanation": entry.get('stateExplanation'),
                "type": entry.get('type')      
            }
        )

facilities_list

[{'id': 1,
  'description': 'zu Gleis 1',
  'operatorname': 'DB Station&Service',
  'state': 'ACTIVE',
  'stateExplanation': 'available',
  'type': 'ELEVATOR'},
 {'id': 1,
  'description': 'zu Gleis 2/3',
  'operatorname': 'DB Station&Service',
  'state': 'ACTIVE',
  'stateExplanation': 'available',
  'type': 'ELEVATOR'},
 {'id': 1,
  'description': 'zu Gleis 6/7',
  'operatorname': 'DB Station&Service',
  'state': 'ACTIVE',
  'stateExplanation': 'available',
  'type': 'ELEVATOR'},
 {'id': 1,
  'description': 'zu Gleis 8/9',
  'operatorname': 'DB Station&Service',
  'state': 'ACTIVE',
  'stateExplanation': 'available',
  'type': 'ELEVATOR'},
 {'id': 1,
  'description': None,
  'operatorname': 'DB Station&Service',
  'state': 'INACTIVE',
  'stateExplanation': 'under construction',
  'type': 'ELEVATOR'},
 {'id': 1,
  'description': None,
  'operatorname': 'DB Station&Service',
  'state': 'INACTIVE',
  'stateExplanation': 'under construction',
  'type': 'ELEVATOR'},
 {'id': 1,
  'descript

In [5]:
df_facilities = pd.DataFrame(facilities_list)

In [5]:
output = open('station_facilities.pkl', 'wb')
pickle.dump(df_facilities, output)
output.close()

NameError: name 'df_facilities' is not defined

### Parking Facilities

In [5]:
pkl_file = open('stations.pkl', 'rb')
df_stations = pickle.load(pkl_file)
pkl_file.close()

In [6]:
parking_facility_json = []

for entry in df_stations['id']:
    try:
        response = requests.get(url_parking_facilities + f'?stopPlaceId={id}', headers=headers)
        json = response.json()
        parking_facility_json.append(json)
    except:
        pass

parking_facility_json

[{'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'_embedded': []},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCode': '429',
  'httpMessage': 'Too Many Requests',
  'moreInformation': 'Assembly Rate Limit exceeded'},
 {'httpCo

In [ ]:
parking_facilities_list = []

for json in facility_json:
    for entry in json.get('facilities', {}):
        parking_facilities_list.append(
            {
                "id": entry['stationnumber'],
                "openingHoursIs24h": entry.get('access', {}).get('openingHours', {}).get('is24h'),
                "openingHoursText": entry.get('access', {}).get('openingHours', {}).get('text'),
                "isOutOfService": entry.get('access', {}).get('outOfService', {}).get('isOutOfService'),
                "iutOfServiceReason": entry.get('access', {}).get('outOfService', {}).get('reason'),
                "hasChargingStation": entry.get('equipment', {}).get('charging', {}).get('hasChargingStation'),
                "hasPrognosis": entry.get('name', [{}])[0].get('name'),
                "stationId": entry.get('station', {}).get('stationId', {}).get('identifier'),
                "stationDistance": entry.get('station', {}).get('distance'),
                "pricesDuration": entry.get('prices', [{}])[0].get('duration'),
                "pricesPrice": entry.get('prices', [{}])[0].get('price'),
            }
        )

parking_facilities_list

In [ ]:
df_parking_facilities = pd.DataFrame(parking_facilities_list)
df_parking_facilities

In [ ]:
output = open('parking_facilities.pkl', 'wb')
pickle.dump(df_parking_facilities, output)
output.close()